# SmolVLA 策略评估 (MuJoCo 环境)

这个 Notebook 在 MuJoCo 仿真环境中评估训练好的 SmolVLA 模型。

## 📊 功能

- 加载训练好的模型
- 在 MuJoCo 环境中运行多个 episodes
- 统计成功率和性能指标
- 可选保存评估视频
- 生成评估报告

## 📈 评估指标

- **平均奖励**: Episode 累计奖励的平均值
- **成功率**: 成功完成任务的 episode 比例
- **Episode 长度**: 完成任务所需的平均步数
- **标准差**: 性能的稳定性指标

---
## 📦 步骤 0: 导入库

In [ ]:
import json
import logging
from pathlib import Path
import time

import gymnasium as gym
import numpy as np
import torch
import matplotlib.pyplot as plt

from lerobot.rl.gym_manipulator import (
    make_robot_env, 
    make_processors, 
    GymManipulatorConfig
)
from lerobot.envs.configs import HILSerlRobotEnvConfig
from lerobot.rl.gym_manipulator import DatasetConfig
from lerobot.processor import TransitionKey, create_transition

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ 库导入成功！")

---
## ⚙️ 步骤 1: 配置参数

In [ ]:
# ========== 模型配置 ==========
MODEL_PATH = "outputs/mujoco_smolvla/checkpoint-20000"  # 训练好的模型路径

# ========== 环境配置 ==========
CONFIG_PATH = "mujoco_so101_config.json"  # 环境配置文件

# ========== 评估参数 ==========
NUM_EPISODES = 10  # 评估的 episodes 数量
MAX_STEPS = 200  # 每个 episode 的最大步数

# ========== 可视化 ==========
RENDER = True  # 是否渲染环境
SAVE_VIDEO = False  # 是否保存视频
VIDEO_DIR = "videos"  # 视频保存目录

# ========== 设备 ==========
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ========== 结果保存 ==========
SAVE_RESULTS = True  # 保存评估结果到 JSON

print("✅ 评估配置:")
print(f"  模型路径: {MODEL_PATH}")
print(f"  Episodes: {NUM_EPISODES}")
print(f"  最大步数: {MAX_STEPS}")
print(f"  渲染: {RENDER}")
print(f"  设备: {DEVICE}")

---
## 🔍 步骤 2: 检查依赖和设备

In [ ]:
# 检查 gym_hil
try:
    import gym_hil
    print("✓ gym_hil 已安装")
except ImportError:
    print("✗ gym_hil 未安装!")
    print("请运行: pip install -e '.[hilserl]'")
    raise

# 检查设备
if DEVICE == "cuda":
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✓ GPU: {gpu_name}")
else:
    print("⚠️ 使用 CPU (推理会较慢)")

print()

---
## 📂 步骤 3: 加载环境配置

In [ ]:
print(f"📂 加载环境配置: {CONFIG_PATH}")

with open(CONFIG_PATH, 'r') as f:
    config_dict = json.load(f)

# 修改配置为评估模式
config_dict["mode"] = None  # 不录制数据
config_dict["device"] = DEVICE
config_dict["env"]["processor"]["observation"]["display_cameras"] = RENDER

# 构建配置对象
env_config = HILSerlRobotEnvConfig(**config_dict['env'])
dataset_config = DatasetConfig(**config_dict['dataset'])

gym_config = GymManipulatorConfig(
    env=env_config,
    dataset=dataset_config,
    mode=config_dict['mode'],
    device=config_dict['device']
)

print("✅ 配置加载成功")

---
## 🧠 步骤 4: 加载训练好的模型

In [ ]:
from lerobot.policies.smolvla import SmolVLAPolicy

print(f"🧠 加载模型: {MODEL_PATH}")

policy = SmolVLAPolicy.from_pretrained(MODEL_PATH)
policy = policy.to(DEVICE)
policy.eval()

param_count = sum(p.numel() for p in policy.parameters()) / 1e6

print(f"✅ 模型加载成功")
print(f"  参数量: {param_count:.1f}M")
print(f"  设备: {DEVICE}")
print()

---
## 🎮 步骤 5: 创建环境和处理器

In [ ]:
print("🎮 创建环境...")

# 创建环境 (评估模式不需要 teleop)
env, _ = make_robot_env(gym_config.env)
print("✓ 环境创建成功")

# 创建处理器
env_processor, action_processor = make_processors(
    env, None, gym_config.env, gym_config.device
)
print("✓ 处理器创建成功")

# 创建视频目录
if SAVE_VIDEO:
    video_dir = Path(VIDEO_DIR)
    video_dir.mkdir(parents=True, exist_ok=True)
    print(f"✓ 视频将保存到: {video_dir}")

print()

---
## 🧪 步骤 6: 定义评估函数

In [ ]:
def evaluate_single_episode(episode_idx):
    """评估单个 episode"""
    
    # Reset environment
    obs, info = env.reset()
    env_processor.reset()
    action_processor.reset()

    # Create initial transition
    transition = create_transition(observation=obs, info=info)
    transition = env_processor(transition)

    episode_reward = 0.0
    episode_step = 0
    done = False
    frames = [] if SAVE_VIDEO else None

    while episode_step < MAX_STEPS and not done:
        # Get action from policy
        with torch.no_grad():
            obs_tensor = transition[TransitionKey.OBSERVATION]
            action = policy.select_action(obs_tensor)

        # Process action
        transition[TransitionKey.ACTION] = action
        transition[TransitionKey.OBSERVATION] = (
            env.get_raw_joint_positions() if hasattr(env, "get_raw_joint_positions") else {}
        )

        processed_transition = action_processor(transition)
        processed_action = processed_transition[TransitionKey.ACTION]

        # Execute action
        obs, reward, terminated, truncated, info = env.step(processed_action)
        done = terminated or truncated

        # Record
        episode_reward += reward
        episode_step += 1

        # Create new transition
        new_transition = create_transition(
            observation=obs,
            action=processed_action,
            reward=reward,
            done=terminated,
            truncated=truncated,
            info=info
        )
        transition = env_processor(new_transition)

        # Render
        if RENDER:
            env.render()

        # Progress
        if episode_step % 10 == 0:
            print(f"\r  Episode {episode_idx + 1}: Step {episode_step}/{MAX_STEPS}, Reward: {episode_reward:.2f}", 
                  end="", flush=True)

    print()  # 换行
    
    # Determine success (需要根据具体任务定义)
    success = episode_reward > 0
    
    return {
        "reward": episode_reward,
        "length": episode_step,
        "success": success
    }

print("✅ 评估函数定义完成")

---
## 🚀 步骤 7: 开始评估

In [ ]:
print("=" * 70)
print(f"🧪 开始评估 ({NUM_EPISODES} episodes)")
print(f"每个 episode 最多 {MAX_STEPS} 步")
print("=" * 70)
print()

episode_results = []

for episode_idx in range(NUM_EPISODES):
    print(f"\n📍 Episode {episode_idx + 1}/{NUM_EPISODES}")
    
    result = evaluate_single_episode(episode_idx)
    episode_results.append(result)
    
    print(f"  完成: {result['length']} 步, "
          f"奖励: {result['reward']:.2f}, "
          f"成功: {result['success']}")

print("\n" + "=" * 70)
print("✅ 评估完成！")
print("=" * 70)

---
## 📊 步骤 8: 统计和可视化结果

In [ ]:
# 提取统计数据
episode_rewards = [r['reward'] for r in episode_results]
episode_lengths = [r['length'] for r in episode_results]
episode_successes = [r['success'] for r in episode_results]

# 计算统计指标
mean_reward = np.mean(episode_rewards)
std_reward = np.std(episode_rewards)
mean_length = np.mean(episode_lengths)
std_length = np.std(episode_lengths)
success_rate = np.mean(episode_successes)

# 打印结果
print("\n" + "=" * 70)
print("📊 评估结果汇总")
print("=" * 70)
print(f"Episodes: {NUM_EPISODES}")
print(f"平均奖励: {mean_reward:.2f} ± {std_reward:.2f}")
print(f"平均长度: {mean_length:.1f} ± {std_length:.1f} 步")
print(f"成功率: {success_rate * 100:.1f}% ({sum(episode_successes)}/{NUM_EPISODES})")
print(f"最高奖励: {np.max(episode_rewards):.2f}")
print(f"最低奖励: {np.min(episode_rewards):.2f}")
print("=" * 70)

In [ ]:
# 可视化结果
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 奖励分布
axes[0].hist(episode_rewards, bins=10, edgecolor='black', alpha=0.7)
axes[0].axvline(mean_reward, color='r', linestyle='--', label=f'平均: {mean_reward:.2f}')
axes[0].set_xlabel('Episode 奖励')
axes[0].set_ylabel('频数')
axes[0].set_title('奖励分布')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Episode 长度分布
axes[1].hist(episode_lengths, bins=10, edgecolor='black', alpha=0.7, color='orange')
axes[1].axvline(mean_length, color='r', linestyle='--', label=f'平均: {mean_length:.1f}')
axes[1].set_xlabel('Episode 长度 (步)')
axes[1].set_ylabel('频数')
axes[1].set_title('Episode 长度分布')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# 成功率
success_count = sum(episode_successes)
fail_count = NUM_EPISODES - success_count
axes[2].bar(['成功', '失败'], [success_count, fail_count], color=['green', 'red'], alpha=0.7)
axes[2].set_ylabel('Episodes')
axes[2].set_title(f'成功率: {success_rate * 100:.1f}%')
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n📈 可视化图表已生成")

---
## 💾 步骤 9: 保存评估结果

In [ ]:
if SAVE_RESULTS:
    results = {
        "model_path": MODEL_PATH,
        "num_episodes": NUM_EPISODES,
        "episode_rewards": episode_rewards,
        "episode_lengths": episode_lengths,
        "episode_successes": episode_successes,
        "mean_reward": float(mean_reward),
        "std_reward": float(std_reward),
        "mean_length": float(mean_length),
        "std_length": float(std_length),
        "success_rate": float(success_rate),
    }
    
    results_file = Path(MODEL_PATH).parent / "eval_results.json"
    with open(results_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"💾 结果已保存到: {results_file}")
else:
    print("⚠️ 未启用结果保存")

---
## 🔍 步骤 10: 分析和建议

In [ ]:
print("\n" + "=" * 70)
print("🔍 性能分析")
print("=" * 70)

# 性能评级
if success_rate >= 0.9:
    rating = "优秀 🌟"
elif success_rate >= 0.7:
    rating = "良好 ✓"
elif success_rate >= 0.5:
    rating = "中等 ○"
else:
    rating = "需要改进 ✗"

print(f"\n性能评级: {rating}")
print(f"成功率: {success_rate * 100:.1f}%")

# 建议
print("\n💡 改进建议:")

if success_rate < 0.7:
    print("  - 采集更多高质量的演示数据 (推荐 100+ episodes)")
    print("  - 增加训练步数 (40k-50k steps)")
    print("  - 调整学习率 (尝试 5e-5 或 2e-4)")
    print("  - 检查数据质量，确保演示流畅自然")

if std_reward > mean_reward * 0.5:
    print("  - 性能不够稳定，建议:")
    print("    * 增加数据多样性")
    * 使用更长的训练时间
    * 考虑数据增强技术")

if mean_length > MAX_STEPS * 0.8:
    print("  - Episode 长度接近上限，可能任务未完成")
    print("    * 增加 MAX_STEPS 或调整任务难度")
    print("    * 优化奖励函数")

if success_rate >= 0.9:
    print("  - 模型表现优秀！可以考虑:")
    print("    * 尝试更复杂的任务")
    print("    * 分享模型到 Hugging Face Hub")
    print("    * 探索 Sim2Real 迁移方法")

print("\n=" * 70)

---
## 🧹 步骤 11: 清理资源

In [ ]:
# 关闭环境
env.close()
print("✅ 环境已关闭")

---
## ✅ 评估完成！

### 总结

您已经成功完成了 SmolVLA 模型的评估：

1. ✅ 加载了训练好的模型
2. ✅ 在 MuJoCo 环境中运行了多个 episodes
3. ✅ 收集了性能指标和统计数据
4. ✅ 可视化了评估结果
5. ✅ 保存了详细的评估报告

### 下一步

根据评估结果，您可以：

1. **如果性能良好** (成功率 > 70%):
   - 尝试更复杂的任务变体
   - 探索 Sim2Real 迁移技术
   - 分享您的模型和经验

2. **如果需要改进** (成功率 < 70%):
   - 返回数据采集阶段，采集更多高质量数据
   - 调整训练超参数
   - 增加训练步数

3. **进阶方向**:
   - 多任务学习
   - 域随机化
   - 迁移到真实 SO101 机械臂

### ⚠️ 重要提醒

**这是 MuJoCo 仿真环境的评估结果**。由于 Sim2Real Gap 的存在，这些结果**不代表**模型在真实 SO101 机械臂上的表现。

如果您想在真实机械臂上使用，请：
1. 在真实环境中采集数据
2. 重新训练或微调模型
3. 参考 `../real_robot/` 目录下的教程

---

**🎉 恭喜完成 MuJoCo + SmolVLA 完整流程！**